<a href="https://colab.research.google.com/github/taddbackus/capstone/blob/main/Working%20RNN/Data_Cleanup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import os
import numpy as np
import pandas as pd

# Geospatial processing packages
import geopandas as gpd

In [62]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Loading dataframe on current active/and closed coal mines

In [63]:
#join in overall info
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/OSMRE_GEOMINE_GEOJSON.geojson')
mines = pd.DataFrame(mines)
mines = mines[['company',
               'coalmine_op_status',
               'mine_name',
               'permit_id',
               'national_id',
               'coal_bed_names',
               'inspectable_unit_status',
                'post_smcra',
               'reported_area',
               'permit_application_type',
               'permit_application_date',
               'permit_approval_date',
               'edit_date',
               'area_mine',
               'contour',
               'mountaintop',
               'steep_slope',
               'highwall',
               'auger',
               'contact']].drop_duplicates()

#replace empty strings with NAN
mines = mines.replace(r'^\s*$', np.nan, regex=True)


In [64]:
### Appending the number of unique companies per permit_id to dataframe
Company_check = mines[['permit_id', 'company']].groupby(['permit_id']).nunique().sort_values('company', ascending=False)
Company_check = Company_check.rename(columns={"company": "Number_company_to_permit_id"})
Company_check[Company_check['Number_company_to_permit_id']>1]
mines = pd.merge(mines, Company_check,  how='left', left_on=['permit_id'], right_on = ['permit_id'])

### Adding in State column to mines df
mines['State'] = mines['national_id'].str.slice(0, 2)
mines['State'].unique()
mines = mines[mines['State']. isin(['KY', 'TN', 'WV'])]
mines['State'] = mines['State'].str.lower()

In [65]:
mines['edit_month'] = mines['edit_date'].astype(str).str.slice(5, 7)
mines['edit_year'] = mines['edit_date'].astype(str).str.slice(0,4)
mines = mines.drop(columns='edit_date')
mines['permit_weekday'] = mines['permit_approval_date'].dt.day_name()
mines['permit_approval_month'] = mines['permit_approval_date'].astype(str).str.slice(5, 7)
mines['permit_approval_year'] = mines['permit_approval_date'].astype(str).str.slice(0,4)
mines = mines.drop(columns='permit_approval_date')
mines['permit_application_year'] =  mines['permit_application_date'].astype(str).str.slice(0,4)
mines = mines.drop(columns='permit_application_date')

In [66]:
mines.columns

Index(['company', 'coalmine_op_status', 'mine_name', 'permit_id',
       'national_id', 'coal_bed_names', 'inspectable_unit_status',
       'post_smcra', 'reported_area', 'permit_application_type', 'area_mine',
       'contour', 'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'State', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year'],
      dtype='object')

In [67]:
#mines = pd.get_dummies(mines, columns=['mountaintop'])
#mines = pd.get_dummies(mines, columns=['area_mine'])
#mines = pd.get_dummies(mines, columns=['highwall'])
#mines = pd.get_dummies(mines, columns=['steep_slope'])
#mines = pd.get_dummies(mines, columns=['coalmine_op_status'])


### Loading dataframes from google earth engine queries

In [68]:
directory_path = '/content/drive/MyDrive/Mines_Data/'
directory_files = os.listdir(directory_path)

df = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['year'] = directory_files[z]
    df_file['year'] = df_file['year'].str.slice(3, 7)
    df_file['end_date'] = df_file['year']+'-07-31'
    df_file['start_date'] = df_file['year']+'-07-01'
    df_file['start_date'] = pd.to_datetime(df_file['start_date'])
    df_file['end_date'] = pd.to_datetime(df_file['end_date'])

    df_file['State'] = directory_files[z]
    df_file['State'] = df_file['State'].str.slice(0, 2)

    df = pd.concat([df, df_file])

    z+=1


### aggregating to averages of land class(the probability) classes

In [69]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file
#419bdf	water, #397d49	trees, #88b053	grass, #7a87c6	flooded_vegetation, #e49635	crops, #dfc35a	shrub_and_scrub, #c4281b	built, #a59b8f	bare, #b39fe1	snow_and_ice

label0 = df[['orig_perm_id', 'Company','mine_id','water', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date','Company']).mean().reset_index()
label1 = df[['orig_perm_id', 'Company','mine_id', 'trees', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label2 = df[['orig_perm_id', 'Company','mine_id', 'grass', 'start_date']].groupby(['orig_perm_id','mine_id','start_date', 'Company']).mean().reset_index()
label3 = df[['orig_perm_id', 'Company','mine_id','flooded_vegetation', 'start_date']].groupby(['orig_perm_id','mine_id', 'start_date', 'Company']).mean().reset_index()
label4 = df[['orig_perm_id', 'Company','mine_id', 'crops', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index().drop_duplicates()
label5 = df[['orig_perm_id', 'Company','mine_id', 'shrub_and_scrub', 'start_date']].groupby(['orig_perm_id', 'start_date', 'mine_id','Company']).mean().reset_index()
label6 = df[['orig_perm_id', 'Company','mine_id', 'built', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label7 = df[['orig_perm_id', 'Company','mine_id', 'bare', 'start_date']].groupby(['orig_perm_id', 'start_date','mine_id', 'Company']).mean().reset_index()
label8 = df[['orig_perm_id', 'Company','mine_id', 'snow_and_ice', 'start_date']].groupby(['orig_perm_id','start_date','mine_id', 'Company']).mean().reset_index()

label0 =label0.drop_duplicates()
label1 =label1.drop_duplicates()
label2 =label2.drop_duplicates()
label3 =label3.drop_duplicates()
label4 =label4.drop_duplicates()
label5 =label5.drop_duplicates()
label6 =label6.drop_duplicates()
label7 =label7.drop_duplicates()
label8 =label8.drop_duplicates()

group_df = df[['orig_perm_id', 'start_date', 'mine_id', 'Company', 'State']].groupby(['orig_perm_id', 'start_date', 'mine_id', 'State']).count().reset_index()
group_df = group_df.drop_duplicates()
group_df = group_df.rename(columns={"Company": "Observations"})
df = pd.merge(df, group_df, on=['orig_perm_id', 'start_date', 'mine_id', 'State'], how='left').drop_duplicates()
df

,ï»¿,id,longitude,latitude,time,water,trees,grass,flooded_vegetation,crops,...,snow_and_ice,label,Company,mine_id,orig_perm_id,year,end_date,start_date,State,Observations
0,3,20230718T161839_20230718T162101_T17SLC,-82.962111,38.040957,1689697971141,0.032271,0.742175,0.033115,0.028178,0.029994,...,0.035817,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
1,9,20230718T161839_20230718T162101_T17SLC,-82.963908,38.042754,1689697971141,0.032551,0.741282,0.033707,0.028543,0.030174,...,0.035345,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
2,15,20230718T161839_20230718T162101_T17SLC,-82.962111,38.042754,1689697971141,0.031584,0.732819,0.034490,0.028255,0.029593,...,0.033416,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
3,19,20230705T160829_20230705T162114_T17SLC,-82.960315,38.042754,1688574175051,0.024305,0.746942,0.039173,0.024682,0.038829,...,0.035887,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
4,21,20230718T161839_20230718T162101_T17SLC,-82.960315,38.042754,1689697971141,0.030797,0.746335,0.035748,0.026772,0.032971,...,0.033623,1.0,B & C ENERGY INC,00000000000000001112,8640180,2023,2023-07-31,2023-07-01,ky,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1358802,12,20170726T160859_20170726T161854_T17SMB,-81.955998,37.766073,1501085934130,0.080749,0.049291,0.047412,0.040685,0.050573,...,0.045043,6.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017,2017-07-31,2017-07-01,wv,10
1358803,13,20170726T160859_20170726T161854_T17SMB,-81.954202,37.766073,1501085934130,0.055003,0.027637,0.028270,0.034314,0.049508,...,0.058603,7.0,BELVA COAL COMPANY,00000000000000000869,S009482,2017,2017-07-31,2017-07-01,wv,10
1358804,1,20170726T160859_20170726T161854_T17SMC,-81.102599,38.278113,1501085934130,0.024015,0.767977,0.030674,0.022576,0.035489,...,0.033115,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017,2017-07-31,2017-07-01,wv,3
1358805,1,20170726T160859_20170726T161854_T17SMC,-81.097209,38.272723,1501085934130,0.024376,0.762291,0.030845,0.022416,0.033265,...,0.030985,1.0,"CHICOPEE COAL COMPANY, INC.",00000000000000000958,S303686,2017,2017-07-31,2017-07-01,wv,3


In [70]:
df = df[['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id', 'State']].drop_duplicates()
df = df.reset_index()
df = df.drop(['index'], axis=1)

### joining aggregated values back into main dataframe

In [71]:
df = pd.merge(df, label0, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label1, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label2, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label3, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label4, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label5, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label6, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label7, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()
df = pd.merge(df, label8, on=['orig_perm_id', 'start_date', 'Company', 'mine_id'], how='left').drop_duplicates()

df.fillna(0, inplace=True)
df

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,crops,shrub_and_scrub,built,bare,snow_and_ice
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,ky,0.031113,0.623011,0.059563,0.032779,0.065345,0.062902,0.035015,0.032943,0.036427
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,0.039489,0.036189,0.033532,0.034271,0.043253
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,ky,0.032205,0.539393,0.055222,0.030944,0.088415,0.041989,0.070858,0.030897,0.037347
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,ky,0.033029,0.735718,0.038324,0.029872,0.033263,0.031299,0.029638,0.032630,0.035219
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,ky,0.030702,0.743186,0.033933,0.027343,0.029957,0.035815,0.029723,0.036228,0.032969
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32655,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,wv,0.031039,0.587889,0.039089,0.025979,0.048085,0.042070,0.107876,0.037235,0.042496
32656,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,wv,0.030835,0.739765,0.032717,0.027254,0.030536,0.037661,0.031239,0.037154,0.032554
32657,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,wv,0.027965,0.710045,0.042984,0.025502,0.038912,0.048706,0.032311,0.033129,0.031919
32658,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,wv,0.036899,0.544176,0.040389,0.030385,0.046551,0.055853,0.064093,0.120093,0.039626


### Checking length before and after joining on permit_id

In [72]:
len(df)

32660

## 22 duplicates using permit_id

### Checking mines with different company names for the same permit_id

In [73]:
rogue_dupe_mine = mines[(mines['permit_id']=='80-156') & (mines['post_smcra']==1.0)]

In [74]:
rogue_dupe_mine

,company,coalmine_op_status,mine_name,permit_id,national_id,coal_bed_names,inspectable_unit_status,post_smcra,reported_area,permit_application_type,...,auger,contact,Number_company_to_permit_id,State,edit_month,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year
302,RICHLAND COAL CO,7,AREA 5,80-156,TN80-156,NaN,NaN,1.0,NaN,NaN,...,NaN,4,2.0,tn,,NaT,NaN,,NaT,NaT


In [75]:
mines = mines[(mines['permit_id']!='80-156')]

In [76]:
mines = mines.append(rogue_dupe_mine)

<ipython-input-76-f3ed2ca40002>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  mines = mines.append(rogue_dupe_mine)


In [77]:
mines.columns

Index(['company', 'coalmine_op_status', 'mine_name', 'permit_id',
       'national_id', 'coal_bed_names', 'inspectable_unit_status',
       'post_smcra', 'reported_area', 'permit_application_type', 'area_mine',
       'contour', 'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'State', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year'],
      dtype='object')

In [78]:
data_check = pd.merge(df, mines,  how='left', left_on=['orig_perm_id', 'State'], right_on = ['permit_id', 'State'])
data_check = data_check.drop_duplicates()
data_check

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,...,highwall,auger,contact,Number_company_to_permit_id,edit_month,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year
0,B & C ENERGY INC,2023-07-01,45,8640180,00000000000000001112,ky,0.031113,0.623011,0.059563,0.032779,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,...,0.0,0.0,3.0,1.0,07,2000,Tuesday,06,1984,NaT
2,HERBERT WELLS,2023-07-01,2,0320080,000000000000000020c7,ky,0.032205,0.539393,0.055222,0.030944,...,0.0,0.0,3.0,0.0,11,1999,Saturday,12,1899,NaT
3,MOUNT VICTORY COAL CO INC,2023-07-01,16,404774X,00000000000000003095,ky,0.033029,0.735718,0.038324,0.029872,...,0.0,0.0,3.0,1.0,07,2000,Monday,06,1975,NaT
4,EVERGREEN MINING INC,2023-07-01,30,8260501,0000000000000000086a,ky,0.030702,0.743186,0.033933,0.027343,...,0.0,0.0,3.0,1.0,05,2004,Monday,03,2001,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32665,LO-MING COAL CORP.,2017-07-01,1,U502397,0000000000000000085a,wv,0.031039,0.587889,0.039089,0.025979,...,NaN,0.0,2.0,1.0,,NaT,Monday,05,1998,NaT
32666,"APPALACHIAN FUELS, LLC",2017-07-01,67,S304188,0000000000000000057b,wv,0.030835,0.739765,0.032717,0.027254,...,NaN,1.0,2.0,1.0,,NaT,Monday,04,1989,NaT
32667,"CHICOPEE COAL COMPANY, INC.",2017-07-01,10,S007385,000000000000000007d5,wv,0.027965,0.710045,0.042984,0.025502,...,NaN,1.0,2.0,1.0,,NaT,Monday,08,1985,NaT
32668,BELVA COAL COMPANY,2017-07-01,10,S009482,00000000000000000869,wv,0.036899,0.544176,0.040389,0.030385,...,NaN,1.0,2.0,1.0,,NaT,Thursday,10,1982,NaT


In [79]:
df_merge = pd.merge(df, mines,  how='left', left_on=['orig_perm_id', 'State'], right_on = ['permit_id', 'State'])

### Creating empty rows for missing years

In [80]:
all_year_mines = df_merge['mine_id'][df_merge['start_date']=='2017-07-01'].unique()

In [81]:
len(all_year_mines)

2176

In [82]:
df_merge.columns

Index(['Company', 'start_date', 'Observations', 'orig_perm_id', 'mine_id',
       'State', 'water', 'trees', 'grass', 'flooded_vegetation', 'crops',
       'shrub_and_scrub', 'built', 'bare', 'snow_and_ice', 'company',
       'coalmine_op_status', 'mine_name', 'permit_id', 'national_id',
       'coal_bed_names', 'inspectable_unit_status', 'post_smcra',
       'reported_area', 'permit_application_type', 'area_mine', 'contour',
       'mountaintop', 'steep_slope', 'highwall', 'auger', 'contact',
       'Number_company_to_permit_id', 'edit_month', 'edit_year',
       'permit_weekday', 'permit_approval_month', 'permit_approval_year',
       'permit_application_year'],
      dtype='object')

In [83]:
df_merge[df_merge['mine_id']=='00000000000000001d20']

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,...,highwall,auger,contact,Number_company_to_permit_id,edit_month,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year
1,RIDNER COAL CO INC,2023-07-01,4,9180010,00000000000000001d20,ky,0.031869,0.698788,0.039705,0.029274,...,0.0,0.0,3.0,1.0,07,2000,Tuesday,06,1984,NaT
6381,RIDNER COAL CO INC,2021-07-01,2,9180010,00000000000000001d20,ky,0.031807,0.735932,0.033805,0.029272,...,0.0,0.0,3.0,1.0,07,2000,Tuesday,06,1984,NaT
9812,RIDNER COAL CO INC,2019-07-01,2,9180010,00000000000000001d20,ky,0.032143,0.735435,0.034380,0.030063,...,0.0,0.0,3.0,1.0,07,2000,Tuesday,06,1984,NaT


In [88]:
date = ['2017-07-01', '2019-07-01', '2021-07-01']
df_clean = pd.DataFrame()
#imputation for missing years by permit_id
keep_columns =['orig_perm_id', 'WaterQuality', 'Max Temperature', 'Precipitation']

for x,i in enumerate(date):
  missing = df_merge['mine_id'][df_merge['start_date']==date[x]].unique()
  df_clean_loop = df_merge.loc[:, df_merge. columns != 'start_date'].query('mine_id not in @missing')

  df_clean_fill = df_clean_loop.groupby(['mine_id', 'Company', 'orig_perm_id']).mean().reset_index()
  df_clean_fill['start_date']=date[x]
  df_clean_fill['start_date'] = pd.to_datetime(df_clean_fill['start_date'])
  df_merge = df_merge.append(df_clean_fill)

  missing = df_merge['mine_id'][df_merge['start_date']==date[x]].unique()
  print('if zero, update succesful: ',len(df_merge.query('mine_id not in @missing')))

### Removing any rows missing 2023

all_year_mines = df_merge['mine_id'][df_merge['start_date']=='07-01-2023'].unique()
df_merge = df_merge.query('mine_id in @all_year_mines')

<ipython-input-88-f576b4056689>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clean_fill = df_clean_loop.groupby(['mine_id', 'Company', 'orig_perm_id']).mean().reset_index()
<ipython-input-88-f576b4056689>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_merge = df_merge.append(df_clean_fill)
<ipython-input-88-f576b4056689>:10: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_clean_fill = df_clean_loop.groupby(['mine_id', 'Company', 'orig_perm_id']).mean().reset_index()
<ipython-input-88-f576b4056689>:13: Fut

if zero, update succesful:  0
if zero, update succesful:  0
if zero, update succesful:  0


<ipython-input-88-f576b4056689>:13: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_merge = df_merge.append(df_clean_fill)


In [ ]:
for i in df_merge.columns:
    if df_merge[i].isna().sum() > 0:
        print('{} has {}% NAs'.format(i,(df_merge[i].isna().sum() / len(df_merge))*100))

company has 62.43924567429201% NAs
coalmine_op_status has 32.04264143607025% NAs
mine_name has 79.60598794634178% NAs
permit_id has 32.04264143607025% NAs
national_id has 32.04264143607025% NAs
coal_bed_names has 68.6799300110168% NAs
inspectable_unit_status has 44.206467500486035% NAs
post_smcra has 32.06208282029681% NAs
reported_area has 100.0% NAs
permit_application_type has 99.98703907718229% NAs
area_mine has 36.72801503467047% NAs
contour has 36.72801503467047% NAs
mountaintop has 36.72801503467047% NAs
steep_slope has 36.72801503467047% NAs
highwall has 45.6386494718424% NAs
auger has 36.72801503467047% NAs
contact has 32.04264143607025% NAs
Number_company_to_permit_id has 32.04264143607025% NAs
edit_month has 32.04264143607025% NAs
edit_year has 32.04264143607025% NAs
permit_weekday has 32.21113343270041% NAs
permit_approval_month has 32.04264143607025% NAs
permit_approval_year has 32.04264143607025% NAs
permit_application_year has 32.04264143607025% NAs


In [89]:
directory_path = '/content/drive/MyDrive/Mines_Data_Clean/Precipitation/'
directory_files = os.listdir(directory_path)

precip = pd.DataFrame()
z=0
for i in directory_files:
    df_file = pd.read_csv(os.path.join(directory_path, i), encoding = 'ISO-8859-1',low_memory=False)
    df_file['start_date_array'] = df_file['start_date_array'].str.split(" ", 1, expand=True)[1]
    df_file['start_date_array'] = df_file['start_date_array'].str.slice(0, 13)
    df_file['start_date'] = pd.to_datetime(df_file['start_date_array'])

    df_file['orig_perm_id'] = df_file['orig_perm_id'].str.split(" ", 1, expand=True)[1]
    df_file['orig_perm_id'] = df_file['orig_perm_id'].str.slice(0, 13)
    df_file['orig_perm_id'] = df_file['orig_perm_id'].str.replace('\nNa', '')

    precip = pd.concat([precip, df_file])

    z+=1

<ipython-input-89-26c763670284>:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_file['start_date_array'] = df_file['start_date_array'].str.split(" ", 1, expand=True)[1]
<ipython-input-89-26c763670284>:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_file['orig_perm_id'] = df_file['orig_perm_id'].str.split(" ", 1, expand=True)[1]
<ipython-input-89-26c763670284>:8: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_file['start_date_array'] = df_file['start_date_array'].str.split(" ", 1, expand=True)[1]
<ipython-input-89-26c763670284>:12: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  df_file['orig_perm_id'] = df_file['orig_perm_i

In [90]:
precip = precip.drop_duplicates()

In [91]:
precip

,ï»¿,WaterQuality,orig_perm_id,start_date_array,start_date,Max Temperature,Precipitation
0,0,0.044621,8640180,2023-07-01,2023-07-01,NaN,NaN
1,1,-0.455063,9180010,2023-07-01,2023-07-01,NaN,NaN
2,2,-0.087868,0320080,2023-07-01,2023-07-01,NaN,NaN
3,3,-0.525961,404774X,2023-07-01,2023-07-01,NaN,NaN
4,4,-0.402627,8260501,2023-07-01,2023-07-01,NaN,NaN
...,...,...,...,...,...,...,...
172,172,-0.499785,8360227,2023-07-01,2023-07-01,299.000000,12.0
173,173,0.023665,0360017,2023-07-01,2023-07-01,298.899994,8.0
174,174,-0.504503,686977X,2023-07-01,2023-07-01,303.600006,3.5
175,175,-0.384532,388574X,2023-07-01,2023-07-01,303.500000,1.7


In [92]:
precip = precip.drop(columns = 'start_date_array')
precip = precip[precip.columns[1:]]

In [93]:
keep_columns =['start_date', 'orig_perm_id', 'WaterQuality', 'Max Temperature', 'Precipitation']
precip = precip[keep_columns]

In [94]:
date = ['2017-07-01', '2019-07-01', '2021-07-01', '2023-07-01']
precip_clean = pd.DataFrame()
#imputation for missing years by permit_id
keep_columns =['orig_perm_id', 'WaterQuality', 'Max Temperature', 'Precipitation']

for x,i in enumerate(date):
  print(date[x])
  missing = precip['orig_perm_id'][precip['start_date']==date[x]]
  #finds permit_id that don't have a row for that year
  precip_clean_loop = precip[keep_columns].query('orig_perm_id not in @missing')

  precip_clean_fill = precip_clean_loop.groupby(['orig_perm_id']).mean().reset_index()
  precip_clean_fill['start_date'] = date[x]
  precip_clean_fill['start_date'] = pd.to_datetime(precip_clean_fill['start_date'])
  precip_clean = precip_clean.append(precip_clean_fill)
  missing = precip_clean['orig_perm_id'][precip_clean['start_date']==date[x]].unique()
  print('if zero, update succesful: ',len(precip_clean.query('orig_perm_id not in @missing')))

  precip_clean = precip_clean.append(precip)

2017-07-01
if zero, update succesful:  0
2019-07-01
if zero, update succesful:  0
2021-07-01
if zero, update succesful:  0
2023-07-01
if zero, update succesful:  0


<ipython-input-94-318adc825d7b>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precip_clean = precip_clean.append(precip_clean_fill)
<ipython-input-94-318adc825d7b>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precip_clean = precip_clean.append(precip)
<ipython-input-94-318adc825d7b>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precip_clean = precip_clean.append(precip_clean_fill)
<ipython-input-94-318adc825d7b>:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  precip_clean = precip_clean.append(precip)
<ipython-input-94-318adc825d7b>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas i

In [95]:
precip_clean['orig_perm_id'] = precip_clean['orig_perm_id'].str.strip()
precip_clean = precip_clean.groupby(['orig_perm_id', 'start_date']).mean().reset_index()

In [96]:
precip_clean[precip_clean['orig_perm_id']=='8640180']

,orig_perm_id,start_date,WaterQuality,Max Temperature,Precipitation
23920,8640180,2017-07-01,-0.459511,299.720001,14.720000
23921,8640180,2019-07-01,-0.682684,304.200012,6.200000
23922,8640180,2021-07-01,-0.740470,295.899994,28.799999
23923,8640180,2023-07-01,0.044621,298.399994,3.600000


In [97]:
len(df_merge)

38662

In [98]:
df_merge_all = pd.merge(df_merge, precip_clean[['WaterQuality', 'orig_perm_id', 'start_date', 'Max Temperature', 'Precipitation']],  how='left', left_on=['orig_perm_id', 'start_date'], right_on = ['orig_perm_id', 'start_date'])

In [99]:
for i in df_merge_all.columns:
    if df_merge_all[i].isna().sum() > 0:
        print('{} has {}% NAs'.format(i,(df_merge_all[i].isna().sum() / len(df_merge_all))*100))

State has 20.174848688634835% NAs
company has 70.01707102581346% NAs
coalmine_op_status has 30.531271015467382% NAs
mine_name has 83.72044901970928% NAs
permit_id has 45.7529356991361% NAs
national_id has 45.7529356991361% NAs
coal_bed_names has 74.99870674046868% NAs
inspectable_unit_status has 42.739640991154104% NAs
post_smcra has 30.551963167968548% NAs
reported_area has 100.0% NAs
permit_application_type has 99.98965392374942% NAs
area_mine has 35.538771920749056% NAs
contour has 35.538771920749056% NAs
mountaintop has 35.538771920749056% NAs
steep_slope has 35.538771920749056% NAs
highwall has 44.04324659872743% NAs
auger has 35.538771920749056% NAs
contact has 30.531271015467382% NAs
Number_company_to_permit_id has 30.531271015467382% NAs
edit_month has 45.7529356991361% NAs
edit_year has 45.7529356991361% NAs
permit_weekday has 45.88743469039367% NAs
permit_approval_month has 45.7529356991361% NAs
permit_approval_year has 45.7529356991361% NAs
permit_application_year has 45.752

In [100]:
file_name = 'all_mines_no_filter_with_precip.csv'
from google.colab import files
df_merge_all.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_merge_all_clean = df_merge_all[df_merge_all['Precipitation'].notna()]

In [ ]:
import numpy as np
from scipy.interpolate import UnivariateSpline
y = df_merge_all_clean['Precipitation']
columns = df_merge_all_clean._get_numeric_data().columns
column_to_exclude = ['Precipitation', 'mine_name','permit_id','trees', 'water', 'snow_and_ice','bare', 'built', 'shrub_and_scrub', 'crops']
columns = list(set(columns) - set(column_to_exclude))

features = df_merge_all_clean[columns]

w = np.isnan(df_merge_all_clean['Precipitation'])
y[w] = 0.
spl = UnivariateSpline(features, y, w=~w)

<ipython-input-59-325a2e8a9b40>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[w] = 0.


ValueError: ignored

In [ ]:
df_merge_all_clean.dropna()

,Company,start_date,Observations,orig_perm_id,mine_id,State,water,trees,grass,flooded_vegetation,...,Number_company_to_permit_id,edit_month,edit_year,permit_weekday,permit_approval_month,permit_approval_year,permit_application_year,WaterQuality,Max Temperature,Precipitation


In [ ]:
df_merge_all_clean = df_merge_all[df_merge_all.notna()]

In [ ]:
from sklearn.preprocessing import StandardScaler
# Make dataframe for model input
X = df_merge_all_clean.loc[:, df_merge_all_clean.columns != 'Precipitation']

#one hot encoding categorical variables for model
cols = X.columns
num_cols = X._get_numeric_data().columns
cat_cols = list(set(cols) - set(num_cols))

#creating dataframe of categorical columns
cat_df = X[cat_cols]
cat_df = pd.get_dummies(cat_df, columns=cat_df.columns)

#creating dataframe of numeric columns
scaler = StandardScaler()
num_df = scaler.fit_transform(X[num_cols])

X=cat_df.join(pd.DataFrame(num_df,columns=X[num_cols].columns, index=X[num_cols].index))
X

/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1047: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1052: RuntimeWarning: invalid value encountered in divide
  T = new_sum / new_sample_count
/usr/local/lib/python3.10/dist-packages/sklearn/utils/extmath.py:1072: RuntimeWarning: invalid value encountered in divide
  new_unnormalized_variance -= correction**2 / new_sample_count


,national_id_KY0070010,national_id_KY0070015,national_id_KY0070017,national_id_KY0070019,national_id_KY0070020,national_id_KY0070022,national_id_KY0070024,national_id_KY0070025,national_id_KY0070026,national_id_KY0070027,...,area_mine,contour,mountaintop,steep_slope,highwall,auger,contact,Number_company_to_permit_id,WaterQuality,Max Temperature
0,0,0,0,0,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.806343,-0.914060
1,0,0,0,0,0,0,0,0,0,0,...,-0.380537,1.702464,-0.161105,-0.137048,-0.025614,-0.365302,0.387246,0.896047,-0.108742,0.766628
2,0,0,0,0,0,0,0,0,0,0,...,-0.380537,-0.587384,-0.161105,-0.137048,-0.025614,-0.365302,0.387246,-1.110443,1.298569,-1.172631
3,0,0,0,0,0,0,0,0,0,0,...,-0.380537,1.702464,-0.161105,-0.137048,-0.025614,-0.365302,0.387246,0.896047,-0.380465,0.895920
4,0,0,0,0,0,0,0,0,0,0,...,2.627864,-0.587384,-0.161105,-0.137048,-0.025614,-0.365302,0.387246,0.896047,0.092226,0.508071
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30857,0,0,0,0,0,0,0,0,0,0,...,-0.380537,-0.587384,-0.161105,-0.137048,NaN,-0.365302,-2.011371,0.896047,NaN,NaN
30858,0,0,0,0,0,0,0,0,0,0,...,-0.380537,1.702464,-0.161105,-0.137048,NaN,2.737461,-2.011371,0.896047,NaN,NaN
30859,0,0,0,0,0,0,0,0,0,0,...,2.627864,1.702464,-0.161105,-0.137048,NaN,2.737461,-2.011371,0.896047,NaN,NaN
30860,0,0,0,0,0,0,0,0,0,0,...,2.627864,1.702464,6.207120,-0.137048,NaN,2.737461,-2.011371,0.896047,NaN,NaN


In [ ]:
y = df_merge_all_clean['Precipitation']
y

0         3.6
1        15.3
2         7.5
3        11.4
4        16.5
         ... 
30857     NaN
30858     NaN
30859     NaN
30860     NaN
30861     NaN
Name: Precipitation, Length: 30862, dtype: float64

In [ ]:
# prompt: create random forest regression model on X_train, y_train, X_test, y_test
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.2,
                                                    random_state=0)

from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=0)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


ValueError: ignored